In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import wandb
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image, ImageDraw
import os

top_layer = 66
api = wandb.Api()
fig, ax = plt.subplots();
DPI = fig.dpi
plt.close()

project_viz="<>/vae_visualizations"
project_scores = "<>/vdvae_analysis"

In [ ]:
shapes ={0: (16, 1, 1),                                                                                                                                                                                             
 1: (16, 1, 1),                                                                                                                                                                                             
 2: (16, 4, 4),                                                                                                                                                                                             
 3: (16, 4, 4),                                                                                                                                                                                             
 4: (16, 4, 4),                                                                                                                                                                                             
 5: (16, 4, 4),                                                                                                                                                                                             
 6: (16, 8, 8),                                                                                                                                                                                             
 7: (16, 8, 8),                                                                                                                                                                                             
 8: (16, 8, 8),                                                                                                                                                                                             
 9: (16, 8, 8),                                                                                                                                                                                             
 10: (16, 8, 8),                                                                                                                                                                                            
 11: (16, 16, 16),                                                                                                                                                                                          
 12: (16, 16, 16),                                                                                                                                                                                          
 13: (16, 16, 16),                                                                                                                                                                                          
 14: (16, 16, 16),
 15: (16, 16, 16),
 16: (16, 16, 16),
 17: (16, 16, 16),
 18: (16, 16, 16),
 19: (16, 16, 16),
 20: (16, 16, 16),
 21: (16, 32, 32),
 22: (16, 32, 32),
 23: (16, 32, 32),
 24: (16, 32, 32),
 25: (16, 32, 32),
 26: (16, 32, 32),
 27: (16, 32, 32),
 28: (16, 32, 32),
 29: (16, 32, 32),
 30: (16, 32, 32),
 31: (16, 32, 32),
 32: (16, 32, 32),
 33: (16, 32, 32),
 34: (16, 32, 32),
 35: (16, 32, 32),
 36: (16, 32, 32),
 37: (16, 32, 32),
 38: (16, 32, 32),
 39: (16, 32, 32),
 40: (16, 32, 32),
 41: (16, 32, 32),
 42: (16, 32, 32),
 43: (16, 64, 64),
 44: (16, 64, 64),
 45: (16, 64, 64),
 46: (16, 64, 64),
 47: (16, 64, 64),
 48: (16, 64, 64),
 49: (16, 64, 64),
 50: (16, 64, 64),
 51: (16, 64, 64),
 52: (16, 64, 64),
 53: (16, 64, 64),
 54: (16, 64, 64),
 55: (16, 64, 64),
 56: (16, 64, 64),
 57: (16, 128, 128),
 58: (16, 128, 128),
 59: (16, 128, 128),
 60: (16, 128, 128),
 61: (16, 128, 128),
 62: (16, 128, 128),
 63: (16, 128, 128),
 64: (16, 128, 128),
 65: (16, 256, 256)}


In [ ]:
import matplotlib

FONTSIZE = 22
TICK_FONT = {
    "fontsize": 20,
    "fontweight": "normal"
}

def modify_font(fontsize=22, ticksize=20, titlesize=24):
    global FONTSIZE
    global TICK_FONT
    FONTSIZE = fontsize
    TICK_FONT = {
        "fontsize": ticksize,
        "fontweight": "normal"
    }

    matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['Verdana']})
    matplotlib.rc('text', usetex=False)
    matplotlib.rc('font', size=FONTSIZE)
    matplotlib.rc('xtick', labelsize=ticksize)
    matplotlib.rc('ytick', labelsize=ticksize)
    matplotlib.rc('figure', titlesize=titlesize)
    matplotlib.rc('axes', titlesize=titlesize)

modify_font(fontsize=22, ticksize=20, titlesize=24)

In [ ]:
def _download(file, path, force_redownload=False):
    full_path = os.path.join(path, file.name)
    if os.path.exists(full_path) and not force_redownload:
        return full_path
    else:
        file.download(path, replace=True)
        return full_path

def download_file(run_id, filename, project=project_scores, force_redownload=False):
    api = wandb.Api()
    run = api.run(f"{project}/{run_id}")
    files = run.files()
    for file in files:
        if file.name == filename:
            return _download(
                file, f"./data/{run_id}/", force_redownload=force_redownload
            )

In [ ]:
score_runs = {
    'qm': "",
    "pm": "",
    "qm_g": "",
    "pm_g": "",
    
}

viz_runs = {
    "attr": "",
    "attr_g": "",
    "sample": "", 
    "pairs": ""
}

regexes = {
    "sample": "media/images/samples.*\.png",
    "var": "media/images/variances_\d.*\.png",
    "var_norm": "media/images/variances_norm.*\.png"
}

In [ ]:
def filter_by_regex(keys, regex):
    r = re.compile(regex)
    return list(filter(r.match, keys))

def sort_images(a):
    return list(sorted(a, key=lambda x: int(x.split("_")[-2])))

In [ ]:
def get_scores(run_id_scores, attr):
    run_scores = api.run(f"{project_scores}/{run_id_scores}")

    files_scores = run_scores.files()
    name2file_scores = {f.name: f for f in files_scores}
    path_score = _download(name2file_scores[f'{attr}.csv'], f"./data/{run_id_scores}/")
    scores = pd.read_csv(path_score)
    scores = scores.set_index("layer_ind")
    return scores

def get_img_attr(run_id_viz, attr, idx=2, temp=0.1):
    run_viz = api.run(f"{project_viz}/{run_id_viz}")
    files = run_viz.files()
    name2file = {f.name: f for f in files}
    lv_points = run_viz.config['lv_points']
    path = _download(name2file[f'{attr}_t{str(temp).replace(".", "_")}_{idx}.png'], f"./data/{run_id_viz}/")
    img = Image.open(path)
    return img, lv_points

def get_img(run_id_viz, regex, idx=0):
    run_viz = api.run(f"{project_viz}/{run_id_viz}")
    files = run_viz.files()
    name2file = {f.name: f for f in files}
    fnames = sort_images(filter_by_regex(name2file.keys(), regex))
    fname = fnames[idx]
    path = _download(name2file[fname], f"./data/{run_id_viz}/")
    img = Image.open(path)
    try:
        lv_points = run_viz.config['lv_points']
        return img, lv_points
    except Exception:
        return img, None

In [ ]:
def grid_to_imgs(img_array, size=128):
    assert img_array.shape[0] % size == 0
    assert img_array.shape[1] % size == 0
    
    n_rows = int(img_array.shape[0]/size)
    n_cols = int(img_array.shape[1]/size)
    
    images = []
    
    for row in range(n_rows):
        img_row = []
        for col in range(n_cols):
            x = img_array[
                (row * size) : (row + 1) * size, 
                (col * size) : (col + 1) * size, 
            ]
            img_row.append(x)
        images.append(img_row)
    return np.array(images)


def imgs_to_grid(images):
    n_rows = images.shape[0]
    n_cols = images.shape[1]
    size = images.shape[2]
    img = np.zeros((size*n_rows, size*n_cols, 3), dtype=images.dtype)
    
    for row in range(n_rows):
        img_row = []
        for col in range(n_cols):
            img[
                (row * size) : (row + 1) * size, 
                (col * size) : (col + 1) * size,
            ] = images[row, col]
    return img

def select_rows_and_cols(img, rows=None, cols=None, size=128):
    img_array = np.array(img)
    imgs = grid_to_imgs(img_array, size=size)
    
    if rows is None:
        rows = list(range(imgs.shape[0]))
    if cols is None:
        cols = list(range(imgs.shape[1]))
        
    img_array = imgs_to_grid(imgs[rows][:,cols])
    return Image.fromarray(img_array)

In [ ]:
def select_for_plot(img, cols, lv_points, layers, layers_from_bottom, size):

    if layers_from_bottom is not None:
        layers = [top_layer - l for l in layers_from_bottom]
    elif layers is not None:
        layers_from_bottom = [top_layer - l for l in layers]

    if layers is None:
        rows = None
        layers = lv_points
        layers_from_bottom = [top_layer - l for l in layers]
    else:
        rows = [lv_points.index(l) for l in layers]

    img = select_rows_and_cols(img, rows, cols, size=size)
    return img, layers_from_bottom

# 1.1.1 Forced

In [ ]:
def plot_forced(
    attr="Male", run_key='attr', 
    layers_from_bottom = None, 
    layers=None, 
    cols = None,
    save_path=None,
    title_suffix="",
    size=256
): 
    img, lv_points = get_img_attr(viz_runs[run_key], attr, idx=2, temp=0.0)
    
    
    img, layers_from_bottom =  select_for_plot(img, cols, lv_points, layers, layers_from_bottom, size)

    plt.figure(figsize=(1.1*img.width/DPI, 1.1*img.height/DPI))
    plt.imshow(img, aspect=1)
    
    yticks = [f"{l}\n{shapes[66-l][0]}x{shapes[66-l][-1]}x{shapes[66-l][-1]}" for l in layers_from_bottom]
    plt.yticks(size/2 + size*np.arange(len(layers_from_bottom)), yticks, **TICK_FONT);
    plt.ylabel(f'latent layer (bottom 1-{top_layer} top)')

    plt.xticks([size*1.2, img.width-size*1.2], [f'Less "{attr}"', f'More "{attr}"'], **TICK_FONT);
    plt.title(f'Forced generation "{attr}"{title_suffix}')
    
    if save_path is not None:
        plt.savefig(save_path, bbox_inches='tight')

In [ ]:
modify_font(fontsize=40, ticksize=40, titlesize=40)

In [ ]:
plot_forced(attr = "Male",
            save_path='fig/forced_male.pdf',  
            cols = [2,4,5,6,7,8,10],
            layers_from_bottom = (66, 64, 63, 57, 33, 26, 23, 18))

In [ ]:
plot_forced(attr = "Bushy_Eyebrows",
            run_key='attr_g',
            title_suffix=" (grouped)",
            save_path='fig/forced_eyebrows.pdf',
#             cols=None,
    cols = [2,4,5,6,7,8,10],
            layers_from_bottom = (64, 63, 59, 52, 39, 26, 23)
           )

In [ ]:
plot_forced(attr = "Pale_Skin",
            run_key='attr_g',
            title_suffix=" (grouped)",
            save_path='fig/pale_skin.pdf',
#             cols=None,
    cols = [2,4,5,6,7,8,10],
            layers_from_bottom = (65, 64,50)
           )

In [ ]:
plot_forced(attr = "Black_Hair",
            run_key='attr_g',
            title_suffix=" (grouped)",
            save_path='fig/black_hair.pdf',
#             cols=None,
    cols = [2,4,5,6,7,8,10],
            layers_from_bottom = (66, 64, 56, 26)
           )

In [ ]:
plot_forced(attr = "Smiling",
            run_key='attr_g',
            title_suffix=" (grouped)",
            save_path='fig/smiling.pdf',
#             cols=None,
    cols = [2,4,5,6,7,8,10],
            layers_from_bottom = (66, 64, 63, 62, 56, 46)
           )

In [ ]:
plot_forced(attr = "Male",
            save_path='fig/forced_male_full_1.pdf',  
            cols = [2,4,5,6,7,8,10],
            layers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14]
           )
plot_forced(attr = "Male",
            save_path='fig/forced_male_full_2.pdf',  
            cols = [2,4,5,6,7,8,10],
            layers = [16, 18, 20, 24, 27, 30, 33, 36, 40, 43, 48, 53]
           )

# 1.1.2 Sampling

In [ ]:
def plot_sampling(
    idx=4,
    layers_from_bottom = None, 
    layers=None, 
    cols = None,
    save_path=None,
    title_suffix="",
    size=128
): 
    img, lv_points = get_img(viz_runs['sample'], regexes['sample'], idx=idx)
    
    
    img, layers_from_bottom =  select_for_plot(img, cols, lv_points, layers, layers_from_bottom, size)

    plt.figure(figsize=(1.1*img.width/DPI, 1.1*img.height/DPI))
    plt.imshow(img, aspect=1)
    
    yticks = [f"{l}\n{shapes[66-l][0]}x{shapes[66-l][-1]}x{shapes[66-l][-1]}" for l in layers_from_bottom]
    plt.yticks(size/2 + size*np.arange(len(layers_from_bottom)), yticks, **TICK_FONT);
    plt.ylabel(f'latent layer (bottom 0-{top_layer} top)')
    plt.xticks([],[])
#     plt.xticks([size*1.2, img.width-size*1.2], [f'Less "{attr}"', f'More "{attr}"'], **TICK_FONT);
#     plt.title(f'Forced generation "{attr}"{title_suffix}')
    
    if save_path is not None:
        plt.savefig(save_path, bbox_inches='tight')

In [ ]:
modify_font(fontsize=44, ticksize=40)
plot_sampling(
    idx=4,
    save_path='fig/sampled_4.pdf', 
    layers_from_bottom=(66, 65, 64, 63, 62, 61, 59, 56, 39, 33, 26, 3),
    cols=list(range(8)),
#     cols=None,
    size=256,
)
modify_font(fontsize=22, ticksize=20)

# 1.1.3 Paired

In [ ]:

modify_font(fontsize=44, ticksize=40)

l1, l2 = 2,3
idx=0
rows= [1,3,5,6,7,8,9]
cols= [0,2,3,4,5,6,7,8,9]
save_path= 'fig/paired_2_3.pdf'
title_suffix=""
l1_b = 66 - l1
l2_b = 66 - l2

img, _ = get_img(viz_runs['pairs'], f"media/images/{l1}_{l2}.*\.png", idx=idx)

img = select_rows_and_cols(img, rows=rows, cols=cols, size=256)

plt.figure(figsize=(1.1*img.width/DPI, 1.1*img.height/DPI))
plt.imshow(img, aspect=1)

plt.xlabel(f'Sampled at layer {l2_b}')
plt.ylabel(f'Sampled at layer {l1_b}')
plt.yticks([],[])
plt.xticks([],[])
plt.title(f'2-layer sampling {l1_b}-{l2_b} {title_suffix}')

if save_path is not None:
    plt.savefig(save_path, bbox_inches='tight')

In [ ]:

modify_font(fontsize=44, ticksize=40)

l1, l2 = 2,7
idx=1
rows= [1,3,5,6,7,8,9]
cols= [0,2,3,4,5,6,7,8,9]
# rows= None
# cols= None
save_path='fig/paired_2_7.pdf'
title_suffix=""
l1_b = 66 - l1
l2_b = 66 - l2
img, _ = get_img(viz_runs['pairs'], f"media/images/{l1}_{l2}.*\.png", idx=idx)

img = select_rows_and_cols(img, rows=rows, cols=cols, size=256)

plt.figure(figsize=(1.1*img.width/DPI, 1.1*img.height/DPI))
plt.imshow(img, aspect=1)

plt.xlabel(f'Sampled at layer {l2_b}')
plt.ylabel(f'Sampled at layer {l1_b}')
plt.yticks([],[])
plt.xticks([],[])
plt.title(f'2-layer sampling {l1_b}-{l2_b} {title_suffix}')

if save_path is not None:
    plt.savefig(save_path, bbox_inches='tight')

In [ ]:

modify_font(fontsize=33, ticksize=30)
l1, l2 = 0,2
idx=1
rows= [1,8,9]
cols= [5,6,9]
# rows= None
# cols= None
save_path='fig/paired_0_2.pdf'
title_suffix=""
l1_b = 66 - l1
l2_b = 66 - l2
img, _ = get_img(viz_runs['pairs'], f"media/images/{l1}_{l2}.*\.png", idx=idx)

img = select_rows_and_cols(img, rows=rows, cols=cols, size=256)

plt.figure(figsize=(1.1*img.width/DPI, 1.1*img.height/DPI))
plt.imshow(img, aspect=1)

plt.xlabel(f'Sampled at layer {l2_b}')
plt.ylabel(f'Sampled at layer {l1_b}')
plt.yticks([],[])
plt.xticks([],[])
plt.title(f'2-layer sampling {l1_b}-{l2_b} {title_suffix}')

if save_path is not None:
    plt.savefig(save_path, bbox_inches='tight')